# Mean-variance analysis with simulated data 
**Camilo A. Garcia Trillos - 2020**

## In this class

    - We are implementing a Markowitz portfolio optimization, after some simulations (loosely based on a version on the blog http://blog.quantopian.com).
    - We learn how to perform an optimisation using the scipy package.
    

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize
np.random.seed(123)

## Markowitz mean-variance analysis

In this class we will start by simulated random data. Later we will learn how to use actual stock data. 

Assume that we have $4$ assets, each with a return series of length $1000$. We can use numpy.random.randn to sample returns from a normal distribution (recall Class A).

In [0]:
## NUMBER OF ASSETS
n_assets = 4

## NUMBER OF OBSERVATIONS
n_obs = 1000

## VECTOR OF MEANS
mu_bar = np.array([0.,0.,0., 0.])
#mu_bar = np.array([0.001,0.002,0.005, 0.008])

## MATRIX OF COVARIANCE

#With this auxSigma matrix, inverting is stable
auxSigma = np.array ( [[1.,0.,0.,0.],[0,1,0,0],[0,0,1,0],[0,0,0,1]]) 

#With this auxSigma matrix, inverting is less stable
#auxSigma = np.array ( [[2,0.,0.,0.],[-0.2,0.7,0,0],[-0.3,0.4,0.7,0],[0.1,0.4,0.4,2.3]]) 

Sigma_exact =auxSigma.T@auxSigma
print (Sigma_exact)

In [0]:
return_vec = np.random.multivariate_normal(mu_bar,Sigma_exact,n_obs)
print (return_vec.shape)

In [0]:
plt.plot(return_vec, alpha=0.3);
plt.xlabel('Time')
plt.ylabel('Returns')

### Random portfolios

Let's generate a couple of random portfolios. Let's assume for the moment that we restrict ourselves to have "long-only" portfolios, that is, the weights are nonnegative.

In [0]:
def rand_weights_pos(n):
    k = np.random.rand(n)
    return k / sum(k)

print(rand_weights_pos(n_assets))
print(rand_weights_pos(n_assets))

Let's next compute the expected return and standard deviation for a portfolio.

In [0]:
def random_portfolio(returns, weights):
    if returns.shape[1] != weights.shape[0]:
        print("Error: MSG by random_portfolio: dimensions don't fit")
        return 0

    w = np.asmatrix(weights)
    p = np.asmatrix(np.mean(returns, axis=0))
    C = np.asmatrix(np.cov(returns.T))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    return mu, sigma

Now, let's plot several such random risk-return profiles:

In [0]:
n_portfolios = 3000
means, stds = np.column_stack([random_portfolio(return_vec, rand_weights_pos(n_assets)) 
                               for _ in range(n_portfolios)])
plt.plot(stds, means,'o', markersize=5)
plt.xlabel('std')
plt.ylabel('mean')
plt.title('Mean and standard deviation of returns of randomly generated portfolios (long-only)')

In [0]:
def rand_weights_unconstrained(n):
    k = np.random.rand(n)*2-1      # Uniformly distributed on [-1,1]
    return k / sum(k)

means_unc, stds_unc = np.column_stack([random_portfolio(return_vec, 
                        rand_weights_unconstrained(n_assets)) for _ in range(2 * n_portfolios)])
plt.plot(stds_unc, means_unc, 'og', markersize=5)
plt.xlabel('std')
plt.ylabel('mean')
plt.title('Mean and standard deviation of returns of randomly generated portfolios')

In [0]:
plt.plot(stds_unc[stds_unc<0.9], means_unc[stds_unc<0.9], 'og', 
         markersize=5, alpha=0.4)
plt.plot(stds, means, 'o', markersize=5, alpha=0.2)
plt.xlabel('std')
plt.ylabel('mean')
plt.title('Mean and standard deviation of returns of randomly generated portfolios')

Let us now check the covariance matrix and find its inverse.

In [0]:
Sigma = np.cov(return_vec.T)
print(Sigma)

Let us check the stability of the matrix with the condioned number

In [0]:
np.linalg.cond(Sigma)

This is not too bad. We could move on without expecting major stability problems.

In [0]:
Sigma_inv = np.linalg.inv(Sigma)

### Markowitz, as done in the lecture notes

In [0]:
def Markowitz_weights(Sigma, mean_returns, mu_p):
    S_inv = np.linalg.inv(Sigma)
    pi_mu = S_inv @ (mean_returns)/sum(S_inv @ mean_returns)
    pi_1 = np.sum(S_inv, axis = 1) / sum(np.sum(S_inv, axis = 1) )
    lambda_demoninator = (mean_returns @ pi_mu) - (mean_returns@pi_1)
    ll = np.array((mu_p - (mean_returns @ pi_1))/lambda_demoninator)
    # previous line: to convert into array in case that mu_p is a single number
    ll.shape=(ll.size,1)
    return pi_mu * ll + pi_1 * (1-ll)

Remark: recall from the lecture that Markowitz needs at least two expected returns to differ. This is not the case via our simulated data. Indeed, any portfolio will have a zero expected return and the equally weighted portfolio will have the smallest variance. Let's just proceed, nevertheless, and see how far we get ... 

In [0]:
Sigma = np.cov(return_vec.T)
mean_returns = np.mean(return_vec, axis = 0)
Markowitz_weights(Sigma, mean_returns, 0.05)

In [0]:
mu_p = np.arange(0.02,0.06,0.002)

In [0]:
Markowitz_weights(Sigma, mean_returns, mu_p)

In [0]:
Mw1 = _

### Markowitz via an optimization algorithm

Next a different that also allows for additional constraints (e.g., nonnegative portfolio weights). 

In [0]:
from scipy.optimize import minimize

minimize?


In [0]:
def Markowitz_weights_opt(Sigma, mean_returns, mu_p, nonnegative = False, print_info = False):
    
    n = Sigma.shape[0]    
    mu_p_vect = np.array(mu_p)
    mu_p_vect.shape = (mu_p_vect.size,)
    # previous lines: to convert into array in case that mu_p is a single number
    portfolios = []
    x0 = np.ones(n)/n

    cost_fun = lambda x: (x@Sigma)@x
    
    for mu_p in mu_p_vect:        
        
        if nonnegative:
            cons = ({'type': 'eq', 'fun': lambda x: x@mean_returns - mu_p},
                    {'type': 'eq', 'fun': lambda x: x@np.ones(n) - 1},
                    {'type': 'ineq', 'fun': lambda x: x})

        else:
            cons = ({'type': 'eq', 'fun': lambda x: x@mean_returns - mu_p},
                    {'type': 'eq', 'fun': lambda x: x@np.ones(n) - 1})
                   
        # Calculate efficient frontier weights using quadratic programming
        res = minimize(cost_fun, x0, method = 'SLSQP', constraints=cons)
        if print_info:
            print(mu_p, res)
        portfolios.append(res.x)
    
    

    # Turning the portfolios into a numpy array
    portfolios = np.array([np.array(p).squeeze() for p in portfolios])
    
    return portfolios

In [0]:
Markowitz_weights_opt(Sigma, mean_returns, 0.012)

In the following yet another implementation. Now, we are using the constraint $\mu^T \pi \geq \mu_p$ instead of $\mu^T \pi = \mu_p$.

In [0]:
def Markowitz_weights_optIneq(Sigma, mean_returns, mu_p_vect, nonnegative = False, print_info = False):
    
    n = Sigma.shape[0]    
    mu_p_vect = np.array(mu_p_vect)
    mu_p_vect.shape = (mu_p_vect.size,)
    # previous lines: to convert into array in case that mu_p is a single number
    portfolios = []
    x0 = np.ones(n)/n

    cost_fun = lambda x: (x@Sigma)@x
    
    for mu_p in mu_p_vect:        
        
        if nonnegative:
            cons = ({'type': 'ineq', 'fun': lambda x: x@mean_returns - mu_p},
                    {'type': 'eq', 'fun': lambda x: x@np.ones(n) - 1},
                    {'type': 'ineq', 'fun': lambda x: x})

        else:
            cons = ({'type': 'ineq', 'fun': lambda x: x@mean_returns - mu_p},
                    {'type': 'eq', 'fun': lambda x: x@np.ones(n) - 1})
                   
        # Calculate efficient frontier weights using quadratic programming
        res = minimize(cost_fun, x0, method = 'SLSQP', constraints=cons)
        if print_info:
            print(mu_p, res)
        portfolios.append(res.x)
        
    # Calculate efficient frontier weights using quadratic programming
    #portfolios = [opt.solvers.qp(S, opt.matrix(np.zeros(n)), G, h, A, opt.matrix([1.0,mu]))['x'] 
    #              for mu in mu_p]

    # Turning the portfolios into a numpy array
    portfolios = np.array([np.array(p).squeeze() for p in portfolios])
    
    return portfolios

In [0]:
Markowitz_weights_opt(Sigma, mean_returns, 0.02)

In [0]:
Markowitz_weights_optIneq(Sigma, mean_returns, 0.02)

In [0]:
Markowitz_weights_opt(Sigma, mean_returns, mu_p)

In [0]:
Markowitz_weights_optIneq(Sigma, mean_returns, mu_p)

In [0]:
Mw1

All three methods therefore yield more or less the same portfolio, which is nice ...  (Markowitz_weights_optIneq only have an inequality constraint and yields results that have further away from the other two approaches).


(If we run again the problem with different means and variances the solution with inequality constraints drift further away. This is simply because those means cannot be attained using only positive portfolios).


In [0]:
portfolio_mu = [mean_returns @ x for x in Mw1]

In [0]:
portfolio_mu

In [0]:
portfolio_sigma = [np.sqrt((x@Sigma)@x) for x in Mw1]

In [0]:
portfolio_sigma

In [0]:
plt.plot(stds, means, 'o', markersize=3)
plt.plot(stds_unc[stds_unc<1], means_unc[stds_unc<1], 'og', 
         markersize=3, alpha=0.4)
plt.ylabel('mean')
plt.xlabel('std')
plt.plot(portfolio_sigma, portfolio_mu, 'ro')
plt.title('Random portfolios and mean-variance frontier')